# CEO-Request Challenge

## 1. Summary of Problem Statement

❓ **How could Olist improve its profit** ❓

#### Revenues  
**Sales fees:** Olist takes a **10% cut** on the product price (excl. freight) of each order delivered  
**Subscription fees:** Olist charges **80 BRL by month** per seller

#### Costs
**Reputation costs** *estimated* per order with bad reviews (<= 3 stars)  

💡 In the long term, bad customer experience has business implications: low repeat rate, immediate customer support cost, refunds or unfavorable word of mouth communication. We will assume that we have an estimate measure of the monetary cost for each bad review:
```python
# review_score: cost(BRL)
{'1 star': 100
'2 stars': 50
'3 stars': 40
'4 stars': 0
'5 stars': 0}
```

**IT costs:** Olist's **total cumulated IT Costs** scale with the square root of the total number of sellers that has ever join the platform, as well as the square root of the total cumulated number of products that was ever sold.  

$IT\_costs = \alpha * \sqrt{n\_sellers} + \beta * \sqrt{n\_products}$  
Olist's data team gave us the following values for these scaling parameters:
- $\alpha = 3157.27$
- $\beta = 978.23$

💡 Both number of sellers to manage and sales transaction are costly for IT systems.  
💡 Yet square roots reflect scale-effects: IT-system are often more efficient as they grow bigger.  
💡 Alpha > Beta means that "Olist has a lower IT Cost with few sellers selling a lot of products rather than the opposite  
- with **1000 sellers** and a total of **100 products** sold, the total IT cost accumulates to 109,624 BRL
- with **100 sellers** and a total of **1000 products** sold, the total IT cost accumulates to 62,507 BRL

Finally, The IT department also told you that since the birth of the marketplace, cumulated IT costs have amounted to **500,000 BRL**.

### Key Findings, so far

- `wait_time` is the most significant factor behind low review scores  
- `wait_time` is made up of seller's `delay_to_carrier` + `carrier_delivery_time`.  
- The latter being outside of Olist's direct control, improving it is not a quick-win recommendation
- On the contrary, a better selection of `sellers` can positively impact the `delay_to_carrier` and reduce the number of bad `review_scores` on Olist.
- Comments of the bad reviews showed that some were linked to the seller or to the product itself.

💡 We recommend you to start with the the guided seller analysis in part 2 below  
💪 But feel free to investigate into other hypothesis instead with part 3

## 2. Should Olist remove under-performing sellers from its marketplace? 🕵🏻
*(recommended)*

To analyze the impact of removing the worse sellers from Olist's marketplace, we will perform a **what-if analysis**

👉 **What would have happened if Olist had never accepted these sellers in the first place?**  

*(In practice, it's hard to know in advance who is a good seller, but let's start with this approach and iterate later).*

### 2.1 Data Preparation

Compute, for each `seller_id`, and cumulated since the beginning:
- the `revenues` it brings
- the `review_costs` associated with all its bad reviews
- the resulting `profits` (revenues - costs)

👉 Write down a step-by-step strategy to create the DataFrame you need


⚠️ Don't start from scratch, update your existing package 😉

Starting from the `Seller` class of your `olist` package:

Edit the `get_training_data` method so that the DataFrame it returns contains the fields:
- `revenues`: sum of subscription and sales fees revenues
- `cost_of_reviews`: sum of costs associated with bad reviews
- `profits`: `revenues` - `cost_of_reviews`

### 2.2 What-if Analysis

👉 Time to perform the actual analysis, here are our steps:  

1️⃣ Create a method that will help us update the IT Costs after removing sellers along with the products they sold

2️⃣ Sort sellers by increasing profits

3️⃣ Remove sellers one by one, starting from the one with the lowest profit.
- For each number of sellers to remove, compute the financial impact on Olist global profits.

4️⃣ Find an optimal number of sellers to remove that maximizes either Olist's profit margin or profit.

In [1]:
from olist.data import Olist
from olist.order import Order
from olist.seller import Seller
from olist.analysis import OlistAnalysis
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt


In [13]:
# Step 1: Compute profits per seller
order = Olist().get_data()
num_orders = order['order_items'].groupby('seller_id')['order_id'].nunique()
revenue = order['order_items'].groupby('seller_id')['price'].sum()
freight_cost = order['order_items'].groupby('seller_id')['freight_value'].sum()
sales_by_seller = pd.concat([num_orders, revenue, freight_cost], axis=1)
sales_by_seller.columns = ['num_orders', 'revenue', 'freight_cost']
profit_by_seller = sales_by_seller['revenue'] - (sales_by_seller['revenue'] * 0.8 + sales_by_seller['freight_cost'])

# Step 2: Sort sellers by increasing profits
sorted_sellers = profit_by_seller.sort_values(ascending=True)
sorted_sellers


seller_id
7c67e1448b00f6e969d365cea6b010ab   -14027.772
1f50f920176fa81dab994f9023523100   -13777.928
a1043bafd471dff536d0c462352beb48   -11463.858
8b321bb669392f5163d04c59e235e066   -10649.222
ea8482cd71df3c1969d7b9473ff13abc   -10109.266
                                      ...    
7d13fca15225358621be4086e1eb0964    14093.744
4869f7a5dfa277a7dca6462dcf3b52b2    25726.456
fa1c13f2614d7b5c4749cbc52fecda94    28765.706
7e93a43ef30c4f03f38b393420bc753a    28964.194
53243585a1d6dc2643021fd1853d8905    31474.580
Length: 3095, dtype: float64

In [17]:
# Step 3: Remove sellers one by one and compute financial impact
profits = []
for i in range(100):
    removed_seller = sorted_sellers.index[i]
    num_orders = order['order_items'].loc[order['order_items']['seller_id']!=removed_seller].groupby('seller_id')['order_id'].nunique()
    revenue = order['order_items'].loc[order['order_items']['seller_id']!=removed_seller].groupby('seller_id')['price'].sum()
    freight_cost = order['order_items'].loc[order['order_items']['seller_id']!=removed_seller].groupby('seller_id')['freight_value'].sum()
    sales_by_seller = pd.concat([num_orders, revenue, freight_cost], axis=1)
    sales_by_seller.columns = ['num_orders', 'revenue', 'freight_cost']
    profit_by_seller = sales_by_seller['revenue'] - (sales_by_seller['revenue'] * 0.8 + sales_by_seller['freight_cost'])
    profits.append(profit_by_seller.sum())

# Step 4: Find optimal number of sellers to remove
optimal_num_sellers = np.argmax(profits) + 1 # add 1 because Python is 0-indexed
profits

[480446.9719999994,
 480197.12799999944,
 477883.0579999994,
 477068.42199999944,
 476528.46599999943,
 473823.02999999945,
 472517.6299999994,
 471950.4919999994,
 471768.2959999994,
 471278.6999999995,
 471230.50599999947,
 470276.64999999944,
 470257.33399999945,
 470053.5139999994,
 470017.01399999944,
 469869.8699999994,
 469719.1739999994,
 469661.9139999994,
 469560.7539999995,
 468961.4999999994,
 468886.3359999994,
 468868.27999999945,
 468677.96599999943,
 468652.97999999946,
 468640.46999999945,
 468619.6899999995,
 468545.8699999994,
 468492.30799999944,
 468282.2739999994,
 468245.79999999946,
 468238.0239999994,
 468211.34399999946,
 468183.0439999994,
 468049.31599999947,
 468019.6119999994,
 468017.3599999994,
 468011.54999999946,
 467988.6219999994,
 467982.45999999944,
 467981.46599999943,
 467946.8499999994,
 467941.4059999994,
 467926.3199999995,
 467912.3499999994,
 467898.3159999994,
 467846.94799999945,
 467843.26799999946,
 467831.38399999944,
 467787.3439999994

## 3. Investigate other Approaches 🕵️
*(optional)*

- Should Olist remove the worst performing products / categories from its marketplace entirely?
- Should Olist remove only consistently underperforming sellers, after it has a honeymoon period of few months?
- Should Olist restrict seller/customer pairs between certain states to avoid delays?
- Should Olist acquire new sellers, with some cost assumptions to be suggested?
- ...


## Your turn!

🧺 Keep this notebook tidy! 

🗣 📊 You will present your insights to your favorite TA at the end of this `Communicate` unit 💪